In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **데이터 살펴보기**

In [2]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [3]:
#경로 설정
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/dacon')

FileNotFoundError: ignored

In [4]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

FileNotFoundError: ignored

In [ ]:
#train 데이터 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


# **전처리**

In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)

train['text']=train['text'].apply(alpha_num)

In [ ]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stopwords:
            final_text.append(i.strip())
    return " ".join(final_text)

# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]

In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num).apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num).apply(remove_stopwords)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])


In [ ]:
X_train

array(['almost choking much much wanted say strange exclamations came lips pole gazed fixedly bundle notes hand looked odin evident perplexity',
       'sister asked suppose',
       'engaged one day walked perusing janes last letter dwelling passages proved jane not written spirits instead surprised mr odin saw looking odin meeting putting away letter immediately forcing smile said',
       ..., 'sincere wellwisher friend sister lucy odin',
       'wanted lend money', 'certainly not occurred said yes like'],
      dtype='<U1433')

# **모델링**

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
#가벼운 NLP모델 생성
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 8s - loss: 1.5660 - accuracy: 0.2762 - val_loss: 1.5535 - val_accuracy: 0.2697
Epoch 2/20
1372/1372 - 8s - loss: 1.4320 - accuracy: 0.3969 - val_loss: 1.3036 - val_accuracy: 0.4526
Epoch 3/20
1372/1372 - 8s - loss: 1.1744 - accuracy: 0.5408 - val_loss: 1.1182 - val_accuracy: 0.5672
Epoch 4/20
1372/1372 - 8s - loss: 1.0205 - accuracy: 0.5997 - val_loss: 1.0100 - val_accuracy: 0.5954
Epoch 5/20
1372/1372 - 8s - loss: 0.9142 - accuracy: 0.6486 - val_loss: 0.9390 - val_accuracy: 0.6380
Epoch 6/20
1372/1372 - 8s - loss: 0.8373 - accuracy: 0.6798 - val_loss: 0.8870 - val_accuracy: 0.6509
Epoch 7/20
1372/1372 - 8s - loss: 0.7752 - accuracy: 0.7081 - val_loss: 0.8405 - val_accuracy: 0.6862
Epoch 8/20
1372/1372 - 8s - loss: 0.7250 - accuracy: 0.7305 - val_loss: 0.8249 - val_accuracy: 0.6828
Epoch 9/20
1372/1372 - 8s - loss: 0.6848 - accuracy: 0.7448 - val_loss: 0.8010 - val_accuracy: 0.6950
Epoch 10/20
1372/1372 - 8s - loss: 0.6482 - accuracy: 0.7620 - val_loss: 0.7887 - 

In [ ]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
pred

array([[1.7437043e-05, 6.9009256e-01, 4.0075846e-02, 2.6946914e-01,
        3.4498752e-04],
       [1.0085062e-01, 6.8909687e-01, 6.2199418e-02, 1.9716928e-02,
        1.2813616e-01],
       [9.9988329e-01, 5.3049307e-06, 2.1452640e-09, 8.5292884e-09,
        1.1135595e-04],
       ...,
       [7.7572295e-05, 9.9991214e-01, 3.2134608e-08, 8.8695542e-06,
        1.4231172e-06],
       [7.4814452e-05, 9.9990869e-01, 2.4786249e-07, 4.8292300e-06,
        1.1378803e-05],
       [9.9934810e-01, 6.1250739e-07, 4.0382545e-07, 1.8201908e-07,
        6.5065757e-04]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,1.743704e-05,6.900926e-01,4.007585e-02,2.694691e-01,3.449875e-04
1,1,1.008506e-01,6.890969e-01,6.219942e-02,1.971693e-02,1.281362e-01
2,2,9.998833e-01,5.304931e-06,2.145264e-09,8.529288e-09,1.113559e-04
3,3,1.856418e-05,4.915246e-10,9.962047e-01,2.093563e-06,3.774721e-03
4,4,9.749085e-01,2.423214e-03,1.449009e-04,2.200382e-02,5.195232e-04
...,...,...,...,...,...,...
19612,19612,4.348929e-07,9.999995e-01,3.135333e-20,7.685614e-11,6.135419e-16
19613,19613,1.491467e-01,1.183721e-05,5.919026e-05,2.775276e-09,8.507822e-01
19614,19614,7.757229e-05,9.999121e-01,3.213461e-08,8.869554e-06,1.423117e-06
19615,19615,7.481445e-05,9.999087e-01,2.478625e-07,4.829230e-06,1.137880e-05


In [ ]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')

In [ ]:
from google.colab import files
files.download('submission.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>